<a href="https://colab.research.google.com/github/matlipson/planets/blob/master/SWAQ_QC_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the code to produce the two quality-controlled csv files as in the TERN repository:

https://portal.tern.org.au/schools-weather-air-sydney-nsw/22077
Cite as:
Hart, M. , Maharaj, A. , Di Virgilio, G. , Ulpiani, G. (2021): Schools Weather and Air Quality (SWAQ) – Quality Controlled Urban Dataset – Sydney (NSW). Version 1.0.0. Terrestrial Ecosystem Research Network (TERN). Dataset. https://doi.org/10.5281/zenodo.5016296

It is written in very plain Python language for use also by beginners in Google Colab, a collaborative environment that runs in Google Drive.
When using the code:
- create a folder in Drive containing the code itself and the dummy_swaq_data.csv dataset. This dataset contains approximately 2-month data in the original format retrieved from the SWAQ Cloud. 
- make sure to update the datapath 
The code comes with a dummy hen processing new data is the file name in the "import SWAQ datafile" Section

In [ ]:
! pip install mpu
! pip install XlsxWriter
import os
import pandas as pd 
import datetime
from calendar import monthrange
import numpy as np
from scipy import stats
import seaborn as sns
import mpu
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/gdrive')
project_folder = "/content/gdrive/My Drive/Colab Notebooks/SWAQ/"
os.chdir(project_folder)

     |████████████████████████████████| 69 kB 2.5 MB/s 
     |████████████████████████████████| 148 kB 3.2 MB/s 
Mounted at /content/gdrive


# ALIGN STATIONS IN TIME AND RESAMPLE TO AVOID GAPS IN TIME

In [ ]:
# function to verify if there is any gap in the timeline. It returns "check passed" if no time misalignment is recorded
# and "WARNING: MISSING DATA" if the timeline is broken
def checkmissing(df):
    df['Time'] = df.index
    # calculate the time difference row by row and add it as a column
    df['deltat'] = (df['Time']-df['Time'].shift()).fillna(pd.Timedelta('0 days'))
    # express the difference in minutes and add it as a column
    df['ans'] = df['deltat'].apply(lambda x: x  / np.timedelta64(1,'m')).astype('int64') % (24*60)
    # print the count of unique values to see how many where different from 20 ± 1 minutes (sampling rate)
    count=(df['ans'].value_counts())
    print (count)
    okvalues=[0,19,20,21]
    total=0
    for z in okvalues:
        if z in count.keys():
            total+=count[z]
    totalok=total/len(df)*100
    if totalok==100:
        str='check passed'
        test=0
    else:
        str='WARNING: MISSING DATA'
        test=1
    print(str)
    df.drop(columns=['Time', 'deltat','ans'],inplace=True)
    return test

In [ ]:
# import SWAQ datafile 
fpath = project_folder+'dummy_swaq_data.csv'
data = pd.read_csv(fpath,index_col=[0],parse_dates=True,dayfirst=True)

In [ ]:
# Use conventional names for columns
oldnames=["AirTemp","AirHum","AirPres","WindDir","WindSpeed"]
newnames=["T","RH","p","wd","ws"]
for scroll in range(len(oldnames)):
    data.columns = data.columns.str.replace(oldnames[scroll], newnames[scroll])

In [ ]:
# Get stations names by looking for unique elements in column name
stations = [col[:col.index("_")] for col in data.columns]
stations=np.asarray(stations)
stations=np.unique(stations)
# Split the SWAQ locations between those measuring meteo only (metonly) and those measuring air quality too (metoaq)
metonly=['DULW','KELL','NARE','TARE','NEWT']
# Get the ramaining locations by subtraction
listations=stations.tolist()
metoaq=list(set(listations)-set(metonly))

In [ ]:
first_st=data.iloc[-1,:].first_valid_index().split('_')[0]
listations.remove(first_st)
listations.insert(0, first_st)

In [ ]:
# Append all subdatasets related to each station into a list, set the Timestamp as datetime and then as index. 
# Use "concat" to merge all dataset along the horizontal axis (axis=1) to align them in time.
mylist=[]
for i in range(len(listations)):
  subdata = data.filter(regex=listations[i])
  timei=(listations[i]+"_Timestamp")
  subdata[timei] = pd.to_datetime(subdata[timei],dayfirst=True)
  subdata = subdata.set_index(subdata[timei])
  subdata = subdata.loc[~subdata.index.duplicated(keep='first')]
  mylist.append(subdata)           
swdataor=pd.concat(mylist,axis=1)
swdataor['Time'] = pd.to_datetime(swdataor.index)
swdataor = swdataor.set_index(pd.DatetimeIndex(swdataor['Time']))

In [ ]:
# Check time gaps
test=checkmissing(swdataor)

0      172210
19      42973
20        744
179         5
4           4
119         1
14          1
5           1
Name: ans, dtype: int64


In [ ]:
# If time gaps, apply resampling on Time column with frequency equal to measurement time step (20 mins). 
# If still gaps send a warning
swdata=swdataor.copy() 
if test==1:
    swdata['Time'] = swdata.index
    swdata=swdata.resample('20Min', on='Time').first()\
           .drop('Time', 1)
    test=checkmissing(swdata)
    if test==1:
        print('WARNING: THE DATA IS STILL SHOWING TIME GAPS. MANUAL CHECK NEEDED')

20    43770
0         1
Name: ans, dtype: int64
check passed


In [ ]:
# write Excel File  
swdata = swdata[swdata.columns.drop(list(swdata.filter(regex='Timestamp')))]
filename=fpath.split('.')[0][-10:-1]+fpath.split('.')[0][-1]
out_path = project_folder+"Data/"+filename+"_aligned&resampled.xlsx"
writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
swdata.to_excel(writer, sheet_name='Sheet1')
writer.save()

# QUALITY CONTROL & FLAGGING

In [ ]:
# Recall aligned and resampled dataset if necessary
datapath = project_folder+"Data/"+filename+"_aligned&resampled.xlsx"
swdataor = pd.read_excel(datapath)
# Set timestamp as dataframe index
swdataor = swdataor.set_index(['Time'])
swdataor.head(3)

,LUDD_NO2,LUDD_SO2,LUDD_CO,LUDD_O3,LUDD_PM25,LUDD_PM10,LUDD_T,LUDD_RH,LUDD_p,LUDD_Rain,LUDD_wd,LUDD_ws,BROO_NO2,BROO_SO2,BROO_CO,BROO_O3,BROO_PM25,BROO_PM10,BROO_T,BROO_RH,BROO_p,BROO_Rain,BROO_wd,BROO_ws,DULW_T,DULW_RH,DULW_p,DULW_Rain,DULW_wd,DULW_ws,DULW_Solarrad,GLEN_NO2,GLEN_SO2,GLEN_CO,GLEN_O3,GLEN_PM25,GLEN_PM10,GLEN_T,GLEN_RH,GLEN_p,...,NARE_p,NARE_Rain,NARE_wd,NARE_ws,NEWT_T,NEWT_RH,NEWT_p,NEWT_Rain,NEWT_wd,NEWT_ws,OEHS_NO2,OEHS_SO2,OEHS_CO,OEHS_O3,OEHS_PM25,OEHS_PM10,OEHS_T,OEHS_RH,OEHS_p,OEHS_Rain,OEHS_wd,OEHS_ws,TARE_T,TARE_RH,TARE_p,TARE_Rain,TARE_wd,TARE_ws,UNSW_NO2,UNSW_SO2,UNSW_CO,UNSW_O3,UNSW_PM25,UNSW_PM10,UNSW_T,UNSW_RH,UNSW_p,UNSW_Rain,UNSW_wd,UNSW_ws
Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-09-01 00:00:00,0.013,0.031,0.058,0.010,2.3,7.3,17.3,51.3,1009.9,0.0,2.0,1.4,0.023,0.025,0.063,-0.001,2.5,4.4,18.6,58.5,1018.3,0.0,205.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015,0.028,0.051,0.015,1.5,3.0,16.1,63.0,1001.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.184,0.046,-0.001,3.5,7.6,17.0,56.4,1016.8,0.0,347.0,0.7,NaN,NaN,NaN,NaN,NaN,NaN,0.021,0.023,0.097,0.012,3.4,8.5,17.5,56.8,1015.1,0.0,262.0,0.6
2019-09-01 00:20:00,0.003,0.025,0.051,0.019,2.3,8.2,18.6,46.8,1009.7,0.0,88.0,1.0,0.023,0.021,0.029,-0.001,2.9,4.5,18.6,60.4,1018.2,0.0,356.0,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010,0.028,0.055,0.020,1.7,3.1,16.6,58.6,1001.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,0.262,0.053,0.002,3.6,9.3,18.0,53.7,1016.5,0.0,60.0,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.021,0.022,0.107,0.014,3.3,8.6,18.0,55.8,1014.9,0.0,270.0,1.0
2019-09-01 00:40:00,0.011,0.023,0.032,0.019,2.0,4.7,19.5,46.3,1009.5,0.0,310.0,0.5,0.018,0.007,0.048,0.000,3.2,4.5,19.5,55.2,1018.0,0.0,324.0,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005,0.026,0.035,0.020,1.5,3.4,18.0,56.0,1000.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013,0.357,0.043,-0.001,3.9,9.0,19.0,50.3,1016.4,0.0,2.0,1.9,NaN,NaN,NaN,NaN,NaN,NaN,0.022,0.017,0.080,0.017,3.4,10.2,18.4,53.4,1014.8,0.0,44.0,0.6


In [ ]:
# Keep track of original dataset and initialize cleaned dataset
swdata = swdataor.copy()
sw_clean = swdataor.copy()

In [ ]:
# Correct NEWT_wd by 180 degrees if before 26 Mar 2021, 11:30 am
mask = swdata.index <= '2021-03-26 11:30:00'
swdata.loc[mask,'NEWT_wd'] = np.where(swdata.loc[mask,'NEWT_wd']+180.0 > 360, swdata.loc[mask,'NEWT_wd']-180.0, swdata.loc[mask,'NEWT_wd']+180.0)
sw_clean.loc[mask,'NEWT_wd'] = np.where(sw_clean.loc[mask,'NEWT_wd']+180.0 > 360, sw_clean.loc[mask,'NEWT_wd']-180.0, sw_clean.loc[mask,'NEWT_wd']+180.0)
# Correct all RH measurements by applying a positive offset of (100-94.7), 
# where 94.7 is the absolute maximum measured on the 2019-31Jan2021 dataset
swdata.loc[:,swdata.filter(regex='_RH').columns]=swdata.loc[:,swdata.filter(regex='_RH').columns]+100-94.7
sw_clean.loc[:,sw_clean.filter(regex='_RH').columns]=sw_clean.loc[:,sw_clean.filter(regex='_RH').columns]+100-94.7
# Remove any column containing "Solarrad" (unknown measurement in Dulwich Hill)
swdata=swdata[swdata.columns.drop(list(swdata.filter(regex='Solarrad')))]
sw_clean=sw_clean[sw_clean.columns.drop(list(sw_clean.filter(regex='Solarrad')))]

In [ ]:
# Create a flag column for each measured parameter
columns=swdata.columns
for col in columns:
    label=col+'_Flags'
    swdata[label]=['' for i in range(swdata.shape[0])]

In [ ]:
##### CONTINUITY TEST: Flag missing rows
##### Set flag columns to "STF4.1" if data is missing
for col in columns:
  varname=col
  flagname=col+'_Flags'
  for ind in swdata.index:
      if pd.isnull(swdata.loc[ind,varname]):
          swdata.loc[ind,flagname] = 'STF4.1;'

In [ ]:
##### FIXED RANGE TESTS: Remove non-physical values
meteovar=['T','RH','p','Rain','ws','wd'];oaqvar=['NO2','SO2','CO','O3','PM25','PM10']
met_lowerbound=[-52.0,0.0,500.0,0.0,0.0,0];oaq_lowerbound=[0.000,0.000,0.0000,0.000,0.000,0.000]
met_upperbound=[60.0,100,1100.0,200,60.0,360];oaq_upperbound=[2000/1000,2000/1000,10000/1000,2000/1000,2000,3276] # 5000 replaced by 3276 after perusing dataset
for st in stations: 
    for var in range(len(meteovar)): 
        varname=st+'_'+meteovar[var]
        flagname=st+'_'+meteovar[var]+'_Flags'
        indices_lo=list(np.where(swdata[varname] < met_lowerbound[var])[0])
        indices_up=list(np.where(swdata[varname] > met_upperbound[var])[0])
        indices=np.concatenate((indices_lo, indices_up)).astype(int)
        swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF2.1;'
        if meteovar[var] in ['RH','Rain','ws','wd']:
            sw_clean.iloc[indices_lo,sw_clean.columns.get_loc(varname)] = met_lowerbound[var]
        if meteovar[var] in ['RH','wd']:
            sw_clean.iloc[indices_up,sw_clean.columns.get_loc(varname)] = met_upperbound[var]
        del indices_up; del indices_lo; del indices
for st in metoaq:        
    for var in range(len(oaqvar)):
        varname=st+'_'+oaqvar[var]
        flagname=st+'_'+oaqvar[var]+'_Flags'
        indices_lo=list(np.where(swdata[varname] < oaq_lowerbound[var])[0])
        indices_up=list(np.where(swdata[varname] > oaq_upperbound[var])[0])
        indices=np.concatenate((indices_lo, indices_up)).astype(int)
        swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF2.1;'
        sw_clean.iloc[indices_lo,sw_clean.columns.get_loc(varname)] = oaq_lowerbound[var]
        sw_clean.iloc[indices_up,sw_clean.columns.get_loc(varname)] = np.nan

In [ ]:
##### INTERNAL CONSISTENCY TESTS: Remove inconsistent values as for inter-parameter associations
for st in stations: 
    varname1=st+'_ws';varname2=st+'_wd'
    flagname1=st+'_ws_Flags';flagname2=st+'_wd_Flags'
    indices=list(np.where((swdata[varname1] == 0) & (swdata[varname2] != 0))[0])
    sw_clean.iloc[indices,sw_clean.columns.get_loc(varname1)] = np.nan
    sw_clean.iloc[indices,sw_clean.columns.get_loc(varname2)] = np.nan
    swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF2.2;'

In [ ]:
##### PERSISTENCE TESTS according to Meek and Hatfield (1994): flag consecutive identical readings over 3 hours
# check if preceding value is the same using .shift() function
for col in columns:
    varname=col[0:13]
    flagname=col[0:13]+'_Flags'
    swdata['same_as_shift'] = swdata[varname].shift() != swdata[varname] 
    for name, group in swdata.groupby(swdata.same_as_shift.cumsum()):
        if (len(group) > 9) & (not ("Rain") in varname):
            swdata.loc[group.index,flagname] = swdata.loc[group.index,flagname] + 'STF3;'
            sw_clean.loc[group.index,varname] = np.nan
    swdata = swdata.drop(columns=['same_as_shift'])

In [ ]:
##### DYNAMIC RANGE TESTS (site-specific extremes): for each station and parameter, verify if 
##### the measured value is an outlier with respect to the monthly dataset of all stations 
#####(Outlier definition: <p25-1.5IQR or >p75+1.5IQR)
# NB: d is a dictionary of outliers. The outmost key is a progressive index for each monthly period:
# By printing d[1] one can access all outliers pertaining to the first month of data, broken down by parameter
allvar = meteovar + oaqvar
allvar=[x for x in allvar if x not in ['Rain','RH','wd']]
allvar = ["_" + par for par in allvar]
idx=1
d = {}
for name, group in swdata.groupby(pd.Grouper(freq="M")):
    d[idx] = {}
    percent=len(group)/(monthrange(name.year, name.month)[1]*24*3)*100
    if percent < 90:
        swdata.loc[group.index,swdata.filter(regex='Flags').columns]=swdata.loc[group.index,swdata.filter(regex='Flags').columns]+"STF4.2;"
        sw_clean.loc[group.index,:]=np.nan
    else:
        for var in allvar:
            d[idx]['Period'] = str(name.month)+'/'+str(name.year)
            d[idx][var[1:]]= {}
            # Find the 75th and 25th percentile of the parameter in object (var) by looking at all stations
            var_cols = [col for col in swdata.columns if (var in col) & (not ("Flags") in col)]
            # Calculate outliers in cleaned dataset (OR remove previous flags???)
            q75,q25 = np.nanpercentile(sw_clean.loc[group.index,var_cols],[75,25])
            # Calculate the interquartile range
            intr_qr = q75-q25
            # Apply the definition of outlier to define the thresholds (max and min)
            max = q75+(1.5*intr_qr)
            min = q25-(1.5*intr_qr)   
            d[idx][var[1:]]['Upper'] = round(max,4)
            d[idx][var[1:]]['Lower'] = round(min,4)
            for i in var_cols:
                flagname=i+'_Flags'
                if idx==1:
                    indices_up=list(np.where(swdata.loc[group.index,i] > max)[0])
                    indices_lo=list(np.where(swdata.loc[group.index,i] < min)[0])
                else:
                    indices_up=list(np.where(swdata.loc[group.index,i] > max)[0])+np.argwhere(swdata.index < group.index[0]).flatten()[-1]+1
                    indices_lo=list(np.where(swdata.loc[group.index,i] < min)[0])+np.argwhere(swdata.index < group.index[0]).flatten()[-1]+1
                indices=np.concatenate((indices_up, indices_lo)).astype(int)
                swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF1.1;'
                del indices_up; del indices_lo; del indices
    idx+=1

In [ ]:
d[17]

{'CO': {'Lower': -0.113, 'Upper': 0.207},
 'NO2': {'Lower': -0.027, 'Upper': 0.085},
 'O3': {'Lower': -0.065, 'Upper': 0.127},
 'PM10': {'Lower': -10.7, 'Upper': 27.7},
 'PM25': {'Lower': -2.0, 'Upper': 4.4},
 'Period': '1/2021',
 'SO2': {'Lower': -0.1625, 'Upper': 0.4575},
 'T': {'Lower': 12.0, 'Upper': 32.0},
 'p': {'Lower': 984.5, 'Upper': 1028.5},
 'ws': {'Lower': -0.95, 'Upper': 2.65}}

In [ ]:
##### STEP TESTS (site-specific extremes): for each station and parameter, verify if 
##### the step from previous measurement is an outlier with respect to the steps of all stations across the same month
#####(Wider outlier definition: <p25-3IQR or >p75+3IQR)
# NB: ds is a dictionary of outliers. The outmost key is a progressive index for each monthly period:
# By printing d[1] one can access all outliers pertaining to the first month of data, broken down by parameter

for col in columns:
    label=col+'_Delta'
    swdata[label]=abs(swdata[col] - swdata[col].shift(1))
    sw_clean[label]=abs(sw_clean[col] - sw_clean[col].shift(1))    
    
allvar = meteovar + oaqvar
allvar=[x for x in allvar if x not in ['Rain','wd']]
allvar = ["_" + par for par in allvar]
idx=1
ds = {}
for name, group in swdata.groupby(pd.Grouper(freq="M")):
    ds[idx] = {}
    percent=len(group)/(monthrange(name.year, name.month)[1]*24*3)*100
    if percent >= 90:
        for var in allvar:
            ds[idx]['Period'] = str(name.month)+'/'+str(name.year)
            ds[idx][var[1:]]= {}
            # Find the 75th and 25th percentile of the parameter in object (var) by looking at all stations
            var_cols = [col for col in swdata.columns if (var in col) & (("Delta") in col) & (not ("Flags") in col)]
            # Calculate outliers in cleaned dataset (OR remove previous flags???)
            q75,q25 = np.nanpercentile(sw_clean.loc[group.index,var_cols],[75,25])
            # Calculate the interquartile range
            intr_qr = q75-q25
            # Apply the definition of outlier to define the thresholds (max)
            max = q75+(3*intr_qr)  
            ds[idx][var[1:]] = round(max,4)
            for i in var_cols:
                flagname=i.replace('_Delta','')+'_Flags'
                if idx==1:
                    indices=list(np.where(swdata.loc[group.index,i] > max)[0])
                else:
                    indices=list(np.where(swdata.loc[group.index,i] > max)[0])+np.argwhere(swdata.index < group.index[0]).flatten()[-1]+1
                # Shall include line to check if indices in not null?
                if len(indices)==0:
                  continue
                else:
                  swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF1.2;'
                del indices
    idx+=1
swdata = swdata[swdata.columns.drop(list(swdata.filter(regex='Delta')))]
sw_clean = sw_clean[sw_clean.columns.drop(list(sw_clean.filter(regex='Delta')))]

In [ ]:
ds[1]

{'CO': 0.059,
 'NO2': 0.013,
 'O3': 0.016,
 'PM10': 15.7,
 'PM25': 4.5,
 'Period': '9/2019',
 'RH': 8.1,
 'SO2': 0.017,
 'T': 1.7,
 'p': 0.9,
 'ws': 2.5}

In [ ]:
##### SPATIAL CONSISTENCY TEST was not included since most accredited algorithms rely on min 6-8 neighbouring stations. 
##### Not the case for SWAQ yet. See Estévez, J., Gavilán, P., & Giráldez, J. V. (2011). 
##### Guidelines on validation procedures for meteorological data from automatic weather stations. Journal of Hydrology, 402(1-2), 144-154.

In [ ]:
##### Use this code to skip the application of combinatorial flags 
# ##### Set data to NaN in cleaned dataset only if both dynamic range test and step test are simultaneously failed
# for col in columns:
#     varname=col[0:13]
#     flagname=col[0:13]+'_Flags'
#     for ind in swdata.index:
#         # the "contains" function is used otherwise cells that have failed also other tests would not be removed
#         if 'STF1.1;STF1.2;' in swdata.loc[ind,flagname]:
#             sw_clean.loc[ind,varname] = np.nan

In [ ]:
##### APPLY COMBINATORIAL FLAGS and set data to NaN in cleaned dataset only if both dynamic range test and step test are simultaneously failed
##### NB: allow 1h 20 mins to complete!
for col in columns:
    varname=col[0:13]
    flagname=col[0:13]+'_Flags'
    for ind in swdata.index:
        if swdata.loc[ind,flagname]=='':
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'STF0;CF0'
        elif 'STF1.1;STF1.2;' in swdata.loc[ind,flagname]:# the "contains" function is used otherwise cells that have failed also other tests would not be removed
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'CF1;'
            sw_clean.loc[ind,varname] = np.nan
        elif ('STF2.1;' in swdata.loc[ind,flagname]) | ('STF2.2;' in swdata.loc[ind,flagname]):
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'CF2'
        elif swdata.loc[ind,flagname]=='STF3':
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'CF3'
        elif ('STF4.1;' in swdata.loc[ind,flagname]) | ('STF4.2;' in swdata.loc[ind,flagname]):
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'CF4'

In [ ]:
# Save cleaned data in current datetime format for analysis
filename0=str(swdata.index[-1]).split('T')[0]+"_Data.csv"
sw_clean.to_csv(r"/content/gdrive/My Drive/Colab Notebooks/SWAQ/Data/"+filename0)
# write csv files with datetime format to ISO 8601
swd=swdata; sw_c=sw_clean
swd.index = swd.index.to_series().apply(datetime.datetime.isoformat)
sw_c.index = sw_c.index.to_series().apply(datetime.datetime.isoformat)
# Get last readings' timestamp and convert it into isoformat then build up filename
filename1=swdata.index[-1].split('T')[0]+"_Raw.csv"
filename2=swdata.index[-1].split('T')[0]+"_Cleaned.csv"
swd.to_csv(r"/content/gdrive/My Drive/Colab Notebooks/SWAQ/Data/"+filename1)
sw_c.to_csv(r"/content/gdrive/My Drive/Colab Notebooks/SWAQ/Data/"+filename2)

# ANALYSIS OF QUALITY CHECKED DATA

In [ ]:
# Recall quality checked dataset if necessary
datapath = "/content/gdrive/My Drive/Colab Notebooks/SWAQ/Data/"+filename1
qcdataor = pd.read_csv(datapath,index_col=[0],parse_dates=True)
qcdataor.head(3)

,LUDD_NO2,LUDD_SO2,LUDD_CO,LUDD_O3,LUDD_PM25,LUDD_PM10,LUDD_T,LUDD_RH,LUDD_p,LUDD_Rain,LUDD_wd,LUDD_ws,BROO_NO2,BROO_SO2,BROO_CO,BROO_O3,BROO_PM25,BROO_PM10,BROO_T,BROO_RH,BROO_p,BROO_Rain,BROO_wd,BROO_ws,DULW_T,DULW_RH,DULW_p,DULW_Rain,DULW_wd,DULW_ws,GLEN_NO2,GLEN_SO2,GLEN_CO,GLEN_O3,GLEN_PM25,GLEN_PM10,GLEN_T,GLEN_RH,GLEN_p,GLEN_Rain,...,NARE_p_Flags,NARE_Rain_Flags,NARE_wd_Flags,NARE_ws_Flags,NEWT_T_Flags,NEWT_RH_Flags,NEWT_p_Flags,NEWT_Rain_Flags,NEWT_wd_Flags,NEWT_ws_Flags,OEHS_NO2_Flags,OEHS_SO2_Flags,OEHS_CO_Flags,OEHS_O3_Flags,OEHS_PM25_Flags,OEHS_PM10_Flags,OEHS_T_Flags,OEHS_RH_Flags,OEHS_p_Flags,OEHS_Rain_Flags,OEHS_wd_Flags,OEHS_ws_Flags,TARE_T_Flags,TARE_RH_Flags,TARE_p_Flags,TARE_Rain_Flags,TARE_wd_Flags,TARE_ws_Flags,UNSW_NO2_Flags,UNSW_SO2_Flags,UNSW_CO_Flags,UNSW_O3_Flags,UNSW_PM25_Flags,UNSW_PM10_Flags,UNSW_T_Flags,UNSW_RH_Flags,UNSW_p_Flags,UNSW_Rain_Flags,UNSW_wd_Flags,UNSW_ws_Flags
Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-09-01 00:00:00,0.013,0.031,0.058,0.010,2.3,7.3,17.3,56.6,1009.9,0.0,2.0,1.4,0.023,0.025,0.063,-0.001,2.5,4.4,18.6,63.8,1018.3,0.0,205.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.015,0.028,0.051,0.015,1.5,3.0,16.1,68.3,1001.3,0.0,...,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF0;CF0,STF1.1;,STF0;CF0,STF2.1;CF2,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0
2019-09-01 00:20:00,0.003,0.025,0.051,0.019,2.3,8.2,18.6,52.1,1009.7,0.0,88.0,1.0,0.023,0.021,0.029,-0.001,2.9,4.5,18.6,65.7,1018.2,0.0,356.0,0.7,NaN,NaN,NaN,NaN,NaN,NaN,0.010,0.028,0.055,0.020,1.7,3.1,16.6,63.9,1001.1,0.0,...,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF0;CF0,STF1.1;STF1.2;CF1;,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0
2019-09-01 00:40:00,0.011,0.023,0.032,0.019,2.0,4.7,19.5,51.6,1009.5,0.0,310.0,0.5,0.018,0.007,0.048,0.000,3.2,4.5,19.5,60.5,1018.0,0.0,324.0,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.005,0.026,0.035,0.020,1.5,3.4,18.0,61.3,1000.9,0.0,...,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF0;CF0,STF1.1;STF1.2;CF1;,STF0;CF0,STF2.1;CF2,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF4.1;CF4,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0,STF0;CF0


In [ ]:
# Keep track of original dataset
qcdata = qcdataor.copy()

In [ ]:
dflag={}
good=[]
for col in columns:
    varname=col[0:13]
    flagname=col[0:13]+'_Flags'
    dflag[varname]={}
    values, counts = np.unique(qcdata[flagname], return_counts=True)
    for idx in range(len(values)):
        dflag[varname][values[idx]] = round(counts[idx]/qcdata.shape[0]*100,0)
    print(varname);print(dflag[varname])
    good.append([varname,dflag[varname]['STF0;CF0']])

LUDD_NO2
{'STF0;CF0': 95.0, 'STF1.1;': 3.0, 'STF1.1;STF1.2;CF1;': 0.0, 'STF1.2;': 1.0, 'STF2.1;CF2': 0.0, 'STF2.1;STF1.2;CF2': 0.0, 'STF3;': 1.0, 'STF4.1;CF4': 0.0}
LUDD_SO2
{'STF0;CF0': 92.0, 'STF1.1;': 1.0, 'STF1.1;STF1.2;CF1;': 0.0, 'STF1.2;': 3.0, 'STF2.1;CF2': 2.0, 'STF2.1;STF1.1;CF2': 1.0, 'STF2.1;STF1.1;STF1.2;CF1;': 0.0, 'STF2.1;STF1.2;CF2': 0.0, 'STF3;': 1.0, 'STF4.1;CF4': 0.0}
LUDD_CO
{'STF0;CF0': 77.0, 'STF1.1;': 3.0, 'STF1.1;STF1.2;CF1;': 3.0, 'STF1.2;': 2.0, 'STF2.1;CF2': 12.0, 'STF2.1;STF1.1;CF2': 1.0, 'STF2.1;STF1.1;STF1.2;CF1;': 1.0, 'STF2.1;STF1.2;CF2': 0.0, 'STF3;': 0.0, 'STF4.1;CF4': 0.0}
LUDD_O3
{'STF0;CF0': 89.0, 'STF1.1;': 3.0, 'STF1.1;STF1.2;CF1;': 0.0, 'STF1.2;': 1.0, 'STF2.1;CF2': 5.0, 'STF2.1;STF1.2;CF2': 0.0, 'STF3;': 2.0, 'STF4.1;CF4': 0.0}
LUDD_PM25
{'STF0;CF0': 61.0, 'STF1.1;': 2.0, 'STF1.1;STF1.2;CF1;': 7.0, 'STF1.2;': 5.0, 'STF2.1;STF1.1;CF2': 0.0, 'STF2.1;STF1.1;STF1.2;CF1;': 0.0, 'STF3;': 25.0, 'STF4.1;CF4': 0.0}
LUDD_PM10
{'STF0;CF0': 89.0, 'STF1.1;':

In [ ]:
## all Rain and ws are 100%
gooddf=pd.DataFrame(good,columns=['Parameter','Good%']).set_index('Parameter')
print('Mean Good%');print(gooddf['Good%'].mean());print('15 smallest Good%');print(gooddf.nsmallest(15, 'Good%'))#;print(gooddf['Good%'].idxmax())

Mean Good%
88.95614035087719
15 smallest Good%
           Good%
Parameter       
UNSW_SO2    42.0
LEPP_PM25   43.0
OEHS_CO     55.0
LUDD_PM25   61.0
KURN_CO     61.0
OEHS_SO2    68.0
BROO_O3     69.0
UNSW_PM25   69.0
UNSW_PM10   70.0
OEHS_ws     72.0
BROO_PM25   73.0
KURN_PM25   73.0
OEHS_PM25   74.0
OEHS_PM10   74.0
GLEN_PM25   75.0


In [ ]:
dflag_m={}
good=[]
countgood=[]
periodlist=[]
for name, group in qcdata.groupby(pd.Grouper(freq="M")):
    count=0
    period=str(name.month)+'/'+str(name.year)
    dflag_m[period]={}
    for col in columns:
        varname=col[0:13]
        flagname=col[0:13]+'_Flags'
        dflag_m[period][varname]={}
        values, counts = np.unique(qcdata.loc[group.index,flagname], return_counts=True)        
        for idx in range(len(values)):
            dflag_m[period][varname][values[idx]] = round(counts[idx],0)
        if 'STF0;CF0' not in values:
            count+=0
            good.append([period,varname,0])
        else:
            count+=dflag_m[period][varname]['STF0;CF0']
            good.append([period,varname,dflag_m[period][varname]['STF0;CF0']])
    countgood.append(count); periodlist.append(period)
zipped_lists = zip(countgood,periodlist)
sorted_zipped_lists = sorted(zipped_lists)
sorted_zipped_lists

[(168685, '9/2019'),
 (189687, '4/2021'),
 (200148, '2/2021'),
 (200567, '3/2021'),
 (218739, '2/2020'),
 (219800, '1/2021'),
 (225220, '12/2020'),
 (225629, '3/2020'),
 (226537, '11/2020'),
 (228192, '6/2020'),
 (228482, '9/2020'),
 (229626, '4/2020'),
 (231989, '11/2019'),
 (232957, '7/2020'),
 (233475, '12/2019'),
 (234338, '8/2020'),
 (234538, '1/2020'),
 (235216, '10/2020'),
 (235934, '5/2020'),
 (239226, '10/2019')]